In [1]:
%%capture
!pip install xvec rioxarray xarray-spatial exactextract memory_profiler

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import earthaccess
import pandas as pd
import geopandas as gpd
import rasterio
import rioxarray
import xvec
import dask
#from xrspatial import zonal_stats
import shapely
assert(earthaccess.login(strategy="netrc").authenticated)

In [3]:
# Only collect polygons with a observer-coded severity value
all_damage = gpd.read_file("../data_working/damage_merged.gdb/", layer="merged")
all_damage = all_damage[~np.isnan(all_damage.PERCENT_MID)]
print(all_damage.shape)
all_damage.head()
bbox_3857 = [-13571302.7073,5853627.9860,-13418428.6507,5966199.5809]
bbox_4326 = rasterio.warp.transform_bounds(3857, 4326, *bbox_3857)

damage_sample = all_damage.cx[bbox_3857[0]:bbox_3857[2], bbox_3857[1]:bbox_3857[3]]
print(damage_sample.shape)

(202967, 6)
(3124, 6)


Search for granules in the Landsat VI collection in this area. Keep all time periods so we can establish pre and post mortality vegetation.

In [4]:
def extract_date(g):
    return pd.to_datetime(g["umm"]["TemporalExtent"]["RangeDateTime"]["BeginningDateTime"][:10], format="%Y-%m-%d")

granules = earthaccess.search_data(
    short_name="HLSL30",
    bounding_box=bbox_4326,
    temporal=("2015-01-01", "2025-01-01")
)

allow_months = [6, 7, 8]

granules = list(filter(lambda g: extract_date(g).month in allow_months, granules))
print(len(granules))

574


In [5]:
gdal_config = {
    'GDAL_HTTP_COOKIEFILE': '~/cookies.txt',
    'GDAL_HTTP_COOKIEJAR': '~/cookies.txt',
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': 'TIF',
    'GDAL_HTTP_UNSAFESSL': 'YES',
    'GDAL_HTTP_MAX_RETRY': '10',
    'GDAL_HTTP_RETRY_DELAY': '0.5',
    'VSI_CACHE': 'FALSE',
    'CPL_VSIL_CURL_NON_CACHED': '/vsicurl/https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/',
    'GDAL_CACHEMAX': 0
}
gdal_env = rasterio.env.Env(**gdal_config)

In [6]:
damage_sample = damage_sample.to_crs(32610)

In [7]:
def granule_zonal_statistics(granule):
    # Open files - but lazily
    data_urls = [item["URL"] for item in granule["umm"]["RelatedUrls"]]
    b5_link = next(filter(lambda x: x.endswith("B05.tif") and x.startswith("https"), data_urls))
    b4_link = next(filter(lambda x: x.endswith("B04.tif") and x.startswith("https"), data_urls))
    qa_link = next(filter(lambda x: x.endswith("Fmask.tif") and x.startswith("https"), data_urls))
    with (
        rioxarray.open_rasterio(b5_link, cache=False).squeeze(drop=True) as b5,
        rioxarray.open_rasterio(b4_link, cache=False).squeeze(drop=True) as b4,
        rioxarray.open_rasterio(qa_link, cache=False).squeeze(drop=True) as fm
    ):
        # Determine the region we need to extract
        featurebox = shapely.geometry.box(*damage_sample.total_bounds)
        rasterbox = shapely.geometry.box(*b5.rio.bounds())
        intersect = shapely.intersection(featurebox, rasterbox)
        
        region = intersect.bounds
        
        # extract
        b5_clip = b5.rio.clip_box(*region)
        b4_clip = b4.rio.clip_box(*region)
        fm_clip = fm.rio.clip_box(*region)
        features = damage_sample.cx[region[0]:region[2], region[1]:region[3]]
        
        # mask out pixels with cloud, cloud shadow, snow/ice, water, and cloud-adjacent
        qa_bitmask = 0b111110
        mask = (fm_clip & qa_bitmask) == 0
        b5_nonan = b5_clip.where(mask)
        b4_nonan = b4_clip.where(mask)
        # calculate ndvi
        ndvi = ((b5_nonan - b4_nonan) / (b5_nonan + b4_nonan)).clip(-1, 1)
        
        # add date - some coordinate other than x/y is necessary for
        # exactextract to work
        ndvi = ndvi.assign_coords(date = extract_date(granule)).expand_dims(dim="date")
        
        # get zonal statistics
        zs_da = ndvi.xvec.zonal_stats(
            features.geometry,
            "x", "y",
            method="exactextract",
            stats=["mean", "stdev", "count"]
        )
    
    # convert to dataset so merging is easier later
    return xr.Dataset({"ndvi": zs_da})\
        .reset_index("geometry", drop=True)\
        .reindex(geometry=zs_da.index)\
        .drop_vars("index")
    

In [8]:
# Make sure it works
import warnings
with gdal_env:
    warnings.filterwarnings("ignore", category=RuntimeWarning, message="Spatial reference system")
    warnings.filterwarnings("ignore", category=FutureWarning, message="Neither osr.UseExceptions()")
    r = granule_zonal_statistics(granules[0])
print(r)

<xarray.Dataset> Size: 3kB
Dimensions:           (date: 1, zonal_statistics: 3, geometry: 107)
Coordinates:
  * date              (date) datetime64[ns] 8B 2015-06-07
  * zonal_statistics  (zonal_statistics) <U5 60B 'mean' 'stdev' 'count'
  * geometry          (geometry) int64 856B 665424 665426 ... 753948 753979
Data variables:
    ndvi              (geometry, zonal_statistics, date) float64 3kB 0.8157 ....


In [9]:
import warnings
import ctypes
import gc

def trim_memory():
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

def worker_driver(granule):
    warnings.filterwarnings("ignore", category=RuntimeWarning, message="Spatial reference system")
    warnings.filterwarnings("ignore", category=FutureWarning, message="Neither osr.UseExceptions()")
    import xvec
    import earthaccess
    assert(earthaccess.login(strategy="netrc").authenticated)
    
    with gdal_env:
        r = granule_zonal_statistics(granule)

    trim_memory()
    gc.collect()
    
    return r
    
from dask.distributed import Client, LocalCluster, progress

# Silence warnings about restarting workers
dask.config.set({"logging.distributed": "error"})

client = Client(
    threads_per_worker=1, 
    n_workers=16, 
    memory_limit="2 GiB", 
    lifetime="5 minutes", 
    lifetime_restart=True,
    lifetime_stagger="5 minutes"
)

lazy_zs = []
for granule in granules:
    this_lazy_zs = dask.delayed(worker_driver)(granule)
    lazy_zs.append(this_lazy_zs)

client

/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38673 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/s-kganz/proxy/38673/status,
Dashboard: /user/s-kganz/proxy/38673/status,Workers: 16
Total threads: 16,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42283,Workers: 0
Dashboard: /user/s-kganz/proxy/38673/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:43269,Total threads: 1
Dashboard: /user/s-kganz/proxy/35329/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:40469,


2025-05-12 21:54:09,559 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:54:34,726 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:55:05,648 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:55:55,902 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:58:10,292 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:58:14,987 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:58:19,094 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:58:30,663 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:58:35,534 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:58:51,937 - distributed.nanny - WARNING - Restarting worker
2025-05-12 21:59:48,068 - distributed.nanny - WARNING - Restarting worker
2025-05-12 22:01:27,278 - distributed.nanny - WARNING - Restarting worker
2025-05-12 22:01:27,759 - distributed.nanny - WARNING - Restarting worker
2025-05-12 22:01:42,379 - distributed.

In [10]:
results = client.compute(lazy_zs, retries=3)

2025-05-12 21:53:59,916 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('worker_driver-fe1ab51f-6499-419d-9714-2b000322d451')" coro=<Worker.execute() done, defined at /srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/worker_state_machine.py:3607>> ended with CancelledError
2025-05-12 21:54:20,433 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('worker_driver-ed76ebe8-f810-45d6-aa73-140d837526d8')" coro=<Worker.execute() done, defined at /srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/worker_state_machine.py:3607>> ended with CancelledError
2025-05-12 21:54:34,291 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-05-12 21:54:59,168 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('worker_driver-daabcd27-eb27-4f87-a1b5-6041999cbdca')" coro=<Worker.execute() done, defined at /srv/conda/en

In [11]:
zs_results = [r.result() for r in results]

2025-05-12 22:04:26,361 - distributed.nanny - ERROR - Worker process died unexpectedly


In [12]:
zs_results[0]

<xarray.Dataset> Size: 3kB
Dimensions:           (date: 1, zonal_statistics: 3, geometry: 107)
Coordinates:
  * date              (date) datetime64[ns] 8B 2015-06-07
  * zonal_statistics  (zonal_statistics) <U5 60B 'mean' 'stdev' 'count'
  * geometry          (geometry) int64 856B 665424 665426 ... 753948 753979
Data variables:
    ndvi              (geometry, zonal_statistics, date) float64 3kB 0.8157 ....

In [13]:
client.close()

In [14]:
zs_dfs = [zs.to_dataframe().reset_index() for zs in zs_results]

In [15]:
zs_df = pd.concat(zs_dfs)
zs_df.head()

,date,zonal_statistics,geometry,ndvi
0,2015-06-07,mean,665424,0.815749
1,2015-06-07,mean,665426,0.858453
2,2015-06-07,mean,665654,0.715021
3,2015-06-07,mean,669034,0.838914
4,2015-06-07,mean,669071,0.841957


In [16]:
# Some duplicates, possibly where granules overlap?
zs_xr = xr.Dataset.from_dataframe(zs_df.groupby(["geometry", "zonal_statistics", "date"]).mean())

In [17]:
zs_xr

<xarray.Dataset> Size: 11MB
Dimensions:           (geometry: 3124, zonal_statistics: 3, date: 144)
Coordinates:
  * geometry          (geometry) int64 25kB 662320 662324 ... 753952 753979
  * zonal_statistics  (zonal_statistics) object 24B 'count' 'mean' 'stdev'
  * date              (date) datetime64[ns] 1kB 2015-06-07 ... 2024-08-27
Data variables:
    ndvi              (geometry, zonal_statistics, date) float64 11MB 61.22 ....

In [18]:
zs_xr.to_netcdf("../data_working/HLSL30_summer_ndvi_cougar_valley.nc")